### Apresentação
Neste trabalho, trabalharemos dados do Kaggle para a Varíola dos Macacos, e verificaremos como os modelos XGBoost e Decision Tree performam em diversos hiperparâmetros.

*link do Kaggle:* https://www.kaggle.com/datasets/muhammad4hmed/monkeypox-patients-dataset

### Etapas
0. Importando Módulos e Dados
1. Exploração e Transformação Inicial
2. Usando XGBoost
3. Usando Decision Tree
4. Considerações finais

### 0. Importando Módulos e Dados

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
!pip install xgboost
from xgboost import XGBClassifier
from sklearn import tree

In [ ]:
df = pd.read_csv('DATA.csv')

In [ ]:
#padronizando nomes de colunas
df.columns = [col.lower().replace(' ','_') for col in df.columns]
df.columns

### 1. Exploração e Transformação Inicial

In [ ]:
df.describe()

In [ ]:
#vamos dropar a colunas Patient_ID por não ter informação relevante
df = df.drop(columns=['patient_id'])

In [ ]:
df.isnull().sum()

não existem nulos neste dataset

In [ ]:
#observando datatypes
df.dtypes

Existem duas colunas do tipo object. Vamos transforma-las em integer.

In [ ]:
#tratando variavel-resposta
### 0 para Negative, 1 para Positive
df['monkeypox'] = df['monkeypox'].replace('Negative', 0).replace('Positive',1).astype(bool)

In [ ]:
#observando systemic illness
print(df['systemic_illness'].unique())
#vamos aplicar o get_dummies() no dataframe
df = pd.get_dummies(df)
df.head(5)

In [ ]:
# transformando todo o dataframe em booleano
df = df.astype(bool)
df.head(5)

In [ ]:
#dividindo em treino e teste
X = df.drop(columns = 'monkeypox')
y = df['monkeypox']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

### 2. Usando XGBoost

In [ ]:
xgb = XGBClassifier(verbosity = 0, random_state = 42, iteration_range = (0,1))
xgb

In [ ]:
params = {
    'booster' : ['gbtree', 'dart'],
    'learning_rate' : [0.3, 0.5, 0.7], #também chamado de eta
    'max_depth' : [3, 5, 7, 11]
}

In [ ]:
xggrid = RandomizedSearchCV(xgb, params, scoring = 'f1', n_jobs = 4, return_train_score = True, n_iter = 10)
xggrid.fit(X_train, y_train)
y_pred = xggrid.predict(X_test)

In [ ]:
print ("PARA XGBOOST")
print('melhores parâmetros:',xggrid.best_params_)
print ('score:', round(xggrid.best_score_**0.5, 3))

In [ ]:
cmxg = confusion_matrix(y_test, y_pred, normalize = 'pred')
ConfusionMatrixDisplay(confusion_matrix = cmxg).plot()

### 3. Usando DecisionTree

In [ ]:
dt = tree.DecisionTreeClassifier(random_state = 42)

In [ ]:
params = {
    'max_depth' : [x for x in range (2,X.shape[1])]
}

In [ ]:
dtgrid = RandomizedSearchCV(dt, params, scoring = 'f1', n_jobs = 4, return_train_score = True, n_iter = 10)
dtgrid.fit(X_train, y_train)
y_pred = dtgrid.predict(X_test)

In [ ]:
print ("PARA DECISION TREE")
print('melhores parâmetros:',dtgrid.best_params_)
print ('score:', round(dtgrid.best_score_**0.5, 3))

In [ ]:
cmdt = confusion_matrix(y_test, y_pred, normalize = 'pred')
ConfusionMatrixDisplay(confusion_matrix = cmdt).plot()

### 4. Considerações Finais
Nota-se, portanto, que o Decision Tree é marginalmente superior que o XGBoost no conjunto de dados escolhido, com o RandomizedSearch afetando hiperparâmetros estabelecidos.

In [ ]:
#plottando árvore de decisão com os melhores hiperparâmetros
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(120,120))
dt = tree.DecisionTreeClassifier(max_depth = 4, random_state=42)
dt.fit(X_train,y_train)
tree.plot_tree(dt, feature_names = df.columns, 
               class_names = ('Negative', 'Positive'),
               filled = True,
               fontsize = 50,
               label = 'root')